# Setup 
## Update Data Directory

In [1]:
# # Update Data/ directory.
# %run ./DownloadData.py

## Imports

In [1]:
import os
import numpy as np
import pandas as pd

## Load DataFrames

In [2]:
df = {}
for data in os.listdir('Data/'):
    if data.endswith('.csv'):
        df[data[:-4]] = pd.read_csv('Data/' + data)
print sorted(df.keys())

['caste', 'caste_grouped', 'caste_percent', 'communicabledisease1', 'communicabledisease1T', 'communicabledisease1_2009T', 'communicabledisease1_2011T', 'communicabledisease1_2013T', 'communicabledisease2', 'communicabledisease2T', 'communicabledisease2_2009T', 'communicabledisease2_2011T', 'hdi2011', 'master', 'master0911', 'master0913', 'master_updated', 'mentalhealth', 'mentalhealthT', 'mentalhealth_2009T', 'mentalhealth_2011T', 'mentalhealth_2013T', 'poverty1', 'poverty1T', 'poverty2', 'poverty2T', 'sexratio', 'simplified_master', 'simplified_master0911', 'simplified_master0913', 'vaccinations', 'vaccinationsT', 'vaccinations_2009T', 'vaccinations_2011T', 'vaccinations_2013T', 'watersanitaion', 'watersanitationT']


# Create DataFrames
## caste_percent
This DataFrame summarizes the percent of the population belonging to each caste by district.

In [3]:
series = []
totalpop = {}
for district in df['caste']['District'].unique():
    caste = df['caste'][(df['caste']['District'] == district)].groupby('Caste/Ethnicity').sum()['In number']
    if district == 'Darchula':
        district = 'Darchaula'
    if district == 'Kavrepalanchok':
        district = 'Kavre'
    series.append((100.0 * (caste / caste.sum())).rename(district))
    totalpop[district] = caste.sum()

df['caste_percent'] = pd.DataFrame(series).fillna(0)
df['caste_percent']['Tot_Pop'] = pd.Series(totalpop).rename('Total_Population')
df['caste_percent'].to_csv('Data/caste_percent.csv')

In [4]:
caste = pd.read_csv('Data/caste.csv')
sg = {'Brahman - Hill':'Hill Brahman', 'Chhetree':'Hill Chhetree', 'Thakuri':'Hill Chhetree', \
      'Dashnami/Sanyasi':'Hill Chhetree', 'Brahman - Tarai':'Madhesi Brahman/Chhetree', \
     'Kayastha':'Madhesi Brahman/Chhetree', 'Rajput':'Madhesi Brahman/Chhetree', 'Amat':'Madhesi Other - OC A', \
     'Badhaee':'Madhesi Other - OC A', 'Baraee':'Madhesi Other - OC A', 'Dev':'Madhesi Other - OC A', \
     'Gaderi/Bhedhar':'Madhesi Other - OC A', 'Hajam/Thakur':'Madhesi Other - OC A', \
     'Haluwai':'Madhesi Other - OC A', 'Kalar':'Madhesi Other - OC A', 'Kalwar':'Madhesi Other - OC A', \
     'Kamar':'Madhesi Other - OC A', 'Kathbaniyan':'Madhesi Other - OC A', 'Koiri/Kushwaha':'Madhesi Other - OC A', \
     'Kurmi':'Madhesi Other - OC A', 'Lohar':'Madhesi Other - OC A', 'Rajbhar':'Madhesi Other - OC A', \
     'Rajdhob':'Madhesi Other - OC A', 'Sonar':'Madhesi Other - OC A', 'Sudhi':'Madhesi Other - OC A', 'Kanu':'Madhesi Other - OC A', \
     'Teli':'Madhesi Other - OC A', 'Yadav':'Madhesi Other - OC A', 'Bin':'Madhesi Other - OC B', \
     'Dhandi':'Madhesi Other - OC B', 'Dhankar/Kharikar':'Madhesi Other - OC B', 'Dhunia':'Madhesi Other - OC B', \
     'Kahar':'Madhesi Other - OC B', 'Kewat':'Madhesi Other - OC B', 'Kori':'Madhesi Other - OC B', \
     'Kumhar':'Madhesi Other - OC B', 'Lodh':'Madhesi Other - OC B', 'Mali':'Madhesi Other - OC B', \
     'Mallaha':'Madhesi Other - OC B', 'Natuwa':'Madhesi Other - OC B', 'Nuniya':'Madhesi Other - OC B', \
     'Nurang':'Madhesi Other - OC B', 'Sarbaria':'Madhesi Other - OC B', 'Badi': 'Hill Dalit', \
     'Damai/Dholi':'Hill Dalit', 'Gaine':'Hill Dalit', 'Kami':'Hill Dalit', 'Sarki':'Hill Dalit', \
     'Bantar/Sardar':'Madhesi Dalit', 'Chamar/Harijan/Ram':'Madhesi Dalit', 'Chidimar':'Madhesi Dalit', \
     'Dhobi':'Madhesi Dalit', 'Dom':'Madhesi Dalit', 'Dusadh/Pasawan/Pasi':'Madhesi Dalit', \
     'Halkhor':'Madhesi Dalit', 'Khatwe':'Madhesi Dalit', 'Musahar':'Madhesi Dalit', 'Tatma/Tatwa':'Madhesi Dalit', \
     'Newar':'Newar', 'Sherpa':'Mtn/Hill Janajati A', 'Yakkha':'Mtn/Hill Janajati A', 'Jirel':'Mtn/Hill Janajati A', \
     'Kulung':'Mtn/Hill Janajati A', 'Yamphu':'Mtn/Hill Janajati A', 'Mewahang':'Mtn/Hill Janajati A', \
     'Gharti/Bhujel':'Mtn/Hill Janajati A', 'Khaling':'Mtn/Hill Janajati A', 'Darai':'Mtn/Hill Janajati A', \
     'Magar':'Mtn/Hill Janajati A', 'Chhantyal/Chhantel':'Mtn/Hill Janajati A', 'Aathpariya':'Mtn/Hill Janajati A', \
     'Bahing':'Mtn/Hill Janajati A', 'Rai':'Mtn/Hill Janajati A', 'Thulung':'Mtn/Hill Janajati A', \
     'Gurung':'Mtn/Hill Janajati A', 'Limbu':'Mtn/Hill Janajati A', 'Lepcha':'Mtn/Hill Janajati A', \
     'Sampang':'Mtn/Hill Janajati A', 'Dura':'Mtn/Hill Janajati A', 'Chamling':'Mtn/Hill Janajati A', \
     'Bantaba':'Mtn/Hill Janajati A', 'Loharung':'Mtn/Hill Janajati A', 'Thakali':'Mtn/Hill Janajati A', \
     'Bhote':'Mtn/Hill Janajati B', 'Bote':'Mtn/Hill Janajati B', 'Brahmu/Baramo':'Mtn/Hill Janajati B', \
     'Byasi/Sauka':'Mtn/Hill Janajati B', 'Chepang/Praja':'Mtn/Hill Janajati B', 'Danuwar':'Mtn/Hill Janajati B', \
     'Dolpo':'Mtn/Hill Janajati B', 'Ghale':'Mtn/Hill Janajati B', 'Hayu':'Mtn/Hill Janajati B', 'Hyolmo':'Mtn/Hill Janajati B', \
     'Kumal':'Mtn/Hill Janajati B', 'Kusunda':'Mtn/Hill Janajati B', 'Lhomi':'Mtn/Hill Janajati B', \
     'Lhopa':'Mtn/Hill Janajati B', 'Majhi':'Mtn/Hill Janajati B', 'Nachhiring':'Mtn/Hill Janajati B', \
     'Pahari':'Mtn/Hill Janajati B', 'Raji':'Mtn/Hill Janajati B', 'Raute':'Mtn/Hill Janajati B', \
     'Sunuwar':'Mtn/Hill Janajati B', 'Tamang':'Mtn/Hill Janajati B', 'Thami':'Mtn/Hill Janajati B', \
     'Topkegola':'Mtn/Hill Janajati B', 'Walung':'Mtn/Hill Janajati B', 'Dhanuk':'Tarai Janajati', \
     'Dhimal':'Tarai Janajati', 'Gangai':'Tarai Janajati', 'Jhangad/Dhagar':'Tarai Janajati', 'Kisan':'Tarai Janajati', \
     'Koche':'Tarai Janajati', 'Meche':'Tarai Janajati', 'Munda':'Tarai Janajati', 'Pattharkatta/Kushwadiya':'Tarai Janajati', \
     'Rajbansi':'Tarai Janajati', 'Satar/Santhal':'Tarai Janajati', 'Tajpuriya':'Tarai Janajati', 'Tharu':'Tarai Janajati', \
     'Musalman':'Musalman', 'Marwadi':'Other (MPB)', 'Punjabi/Shikh':'Other (MPB)', 'Bangali':'Other (MPB)', \
     'Dalit Others':'Other undefined', 'Janajati Others':'Other undefined', 'Undefined Others':'Other undefined', \
     'Terai Others':'Other undefined', 'Others':'Other undefined', 'Foreigner':'Foreigner', 'Khawas':'Other undefined'}
caste['social_groups'] = caste['Caste/Ethnicity'].map(sg)
grouped = pd.DataFrame()
grouped['district'] = caste['District']
grouped['unique'] = caste['social_groups']
grouped['number'] = caste['In number']
grouped = pd.DataFrame(grouped['number'].groupby([grouped['district'], grouped['unique']]).sum())
grouped = grouped.reset_index()
grouped = grouped.pivot(index='district', columns='unique', values='number')
grouped = grouped.reset_index()
grouped = grouped.fillna(0)
grouped['Total Pop Census'] = grouped.sum(axis=1)
grouped.iloc[:,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]] = \
   grouped.iloc[:,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]].div(grouped['Total Pop Census'],axis=0)
grouped = grouped.set_index('district')
grouped.index = grouped.index.rename('District ')
grouped.to_csv('Data/caste_grouped.csv')

## Poverty

In [5]:
poverty = df['poverty1'][df['poverty1']['Year AD'] == 2011]
series = []
for district in poverty['District'].unique():
    indicators = poverty[(poverty['District'] == district)].groupby('Indicators').sum()['Value']
    series.append(indicators.rename(district).T)

df['poverty1T'] = pd.DataFrame(series)
df['poverty1T'].to_csv('Data/poverty1T.csv')

series = []
poverty = df['poverty2']

for district in poverty['District'].unique():
    subgroups = poverty[(poverty['District'] == district)].groupby('Sub Group').sum()['Value']
    series.append(subgroups.rename(district).T)

df['poverty2T'] = pd.DataFrame(series)
df['poverty2T'].to_csv('Data/poverty2T.csv')

## vaccinationsT

In [6]:
vacc = df['vaccinations'][df['vaccinations']['Year AD'] == '2009/10']
series = []

for district in vacc['District'].unique():
    vaccines = vacc[vacc['District'] == district].groupby('Vaccines').sum()
    vaccines.columns = [district]
    series.append(vaccines.T.ix[0])
    
df['vaccinations_2009T'] = pd.DataFrame(series)
df['vaccinations_2009T'].to_csv('Data/vaccinations_2009T.csv')

In [7]:
vacc = df['vaccinations'][df['vaccinations']['Year AD'] == '2011/12']
series = []

for district in vacc['District'].unique():
    vaccines = vacc[vacc['District'] == district].groupby('Vaccines').sum()
    vaccines.columns = [district]
    series.append(vaccines.T.ix[0])
    
df['vaccinations_2011T'] = pd.DataFrame(series)
df['vaccinations_2011T'].to_csv('Data/vaccinations_2011T.csv')

In [8]:
vacc = df['vaccinations'][df['vaccinations']['Year AD'] == '2013/14']
series = []

for district in vacc['District'].unique():
    vaccines = vacc[vacc['District'] == district].groupby('Vaccines').sum()
    vaccines.columns = [district]
    series.append(vaccines.T.ix[0])
    
df['vaccinations_2013T'] = pd.DataFrame(series)
df['vaccinations_2013T'].to_csv('Data/vaccinations_2013T.csv')

## watersanitationT

In [9]:
df['watersanitaion']
series = []

for district in df['watersanitaion']['Districts'].unique():
    sani = df['watersanitaion'][df['watersanitaion']['Districts'] == district].groupby('Sub group').sum()
    sani.columns = [district]
    series.append(sani.T.ix[0])
df['watersanitationT'] = pd.DataFrame(series)
df['watersanitationT'].to_csv('Data/watersanitationT.csv')

## mentalhealthT

In [10]:
mh = df['mentalhealth'][df['mentalhealth']['Year AD'] == '2009/10']
series = []

for district in mh['District'].unique():
    mental = mh[mh['District'] == district].groupby('Mental problem').sum()
    mental.columns = [district]
    series.append(mental.T.ix[0])

df['mentalhealth_2009T'] = pd.DataFrame(series)
df['mentalhealth_2009T'].to_csv('Data/mentalhealth_2009T.csv')

In [11]:
mh = df['mentalhealth'][df['mentalhealth']['Year AD'] == '2011/12']
series = []

for district in mh['District'].unique():
    mental = mh[mh['District'] == district].groupby('Mental problem').sum()
    mental.columns = [district]
    series.append(mental.T.ix[0])

df['mentalhealth_2011T'] = pd.DataFrame(series)
df['mentalhealth_2011T'].to_csv('Data/mentalhealth_2011T.csv')

In [12]:
mh = df['mentalhealth'][df['mentalhealth']['Year AD'] == '2013/14']
series = []

for district in mh['District'].unique():
    mental = mh[mh['District'] == district].groupby('Mental problem').sum()
    mental.columns = [district]
    series.append(mental.T.ix[0])

df['mentalhealth_2013T'] = pd.DataFrame(series)
df['mentalhealth_2013T'].to_csv('Data/mentalhealth_2013T.csv')

## Communicable Disease

In [13]:
cd1 = df['communicabledisease1'][df['communicabledisease1']['Year AD'] == '2009/10']
series = []

for district in cd1['District'].unique():
    disease1 = cd1[cd1['District'] == district].groupby('Communicable Diseases').sum()
    disease1.columns = [district]
    series.append(disease1.T.ix[0])

df['communicabledisease1_2009T'] = pd.DataFrame(series)
df['communicabledisease1_2009T'].to_csv('Data/communicabledisease1_2009T.csv')

cd2 = df['communicabledisease2'][df['communicabledisease2']['Year AD'] == '2009/10']
series = []

for district in cd2['District'].unique():
    disease2 = cd2[cd2['District'] == district].groupby('Diseases').sum()
    disease2.columns = [district]
    series.append(disease2.T.ix[0])

df['communicabledisease2_2009T'] = pd.DataFrame(series)
df['communicabledisease2_2009T'].to_csv('Data/communicabledisease2_2009T.csv')

In [14]:
cd1 = df['communicabledisease1'][df['communicabledisease1']['Year AD'] == '2011/12']
series = []

for district in cd1['District'].unique():
    disease1 = cd1[cd1['District'] == district].groupby('Communicable Diseases').sum()
    disease1.columns = [district]
    series.append(disease1.T.ix[0])

df['communicabledisease1_2011T'] = pd.DataFrame(series)
df['communicabledisease1_2011T'].to_csv('Data/communicabledisease1_2011T.csv')

cd2 = df['communicabledisease2'][df['communicabledisease2']['Year AD'] == '2011/12']
series = []

for district in cd2['District'].unique():
    disease2 = cd2[cd2['District'] == district].groupby('Diseases').sum()
    disease2.columns = [district]
    series.append(disease2.T.ix[0])

df['communicabledisease2_2011T'] = pd.DataFrame(series)
df['communicabledisease2_2011T'].to_csv('Data/communicabledisease2_2011T.csv')

In [15]:
cd1 = df['communicabledisease1'][df['communicabledisease1']['Year AD'] == '2013/14']
series = []

for district in cd1['District'].unique():
    disease1 = cd1[cd1['District'] == district].groupby('Communicable Diseases').sum()
    disease1.columns = [district]
    series.append(disease1.T.ix[0])

df['communicabledisease1_2013T'] = pd.DataFrame(series)
df['communicabledisease1_2013T'].to_csv('Data/communicabledisease1_2013T.csv')

In [16]:
print sorted(df.keys())

['caste', 'caste_grouped', 'caste_percent', 'communicabledisease1', 'communicabledisease1T', 'communicabledisease1_2009T', 'communicabledisease1_2011T', 'communicabledisease1_2013T', 'communicabledisease2', 'communicabledisease2T', 'communicabledisease2_2009T', 'communicabledisease2_2011T', 'hdi2011', 'master', 'master0911', 'master0913', 'master_updated', 'mentalhealth', 'mentalhealthT', 'mentalhealth_2009T', 'mentalhealth_2011T', 'mentalhealth_2013T', 'poverty1', 'poverty1T', 'poverty2', 'poverty2T', 'sexratio', 'simplified_master', 'simplified_master0911', 'simplified_master0913', 'vaccinations', 'vaccinationsT', 'vaccinations_2009T', 'vaccinations_2011T', 'vaccinations_2013T', 'watersanitaion', 'watersanitationT']


In [17]:
df['hdi2011'] = df['hdi2011'].set_index(['District '])
df['sexratio'] = df['sexratio'].set_index('District ')

# Create Master 2011-2012 Dataset

In [20]:
keys = []
for elem in df.keys():
    keys.append(elem)
print keys

['poverty1', 'poverty2', 'master0913', 'master0911', 'communicabledisease1_2009T', 'hdi2011', 'communicabledisease1_2013T', 'communicabledisease2T', 'communicabledisease1_2011T', 'sexratio', 'mentalhealth', 'communicabledisease2_2009T', 'simplified_master0911', 'simplified_master0913', 'master', 'watersanitaion', 'poverty2T', 'mentalhealth_2009T', 'vaccinations_2009T', 'poverty1T', 'simplified_master', 'communicabledisease2', 'communicabledisease1', 'caste_percent', 'master_updated', 'mentalhealth_2011T', 'vaccinations', 'vaccinationsT', 'communicabledisease1T', 'mentalhealth_2013T', 'vaccinations_2011T', 'mentalhealthT', 'vaccinations_2013T', 'watersanitationT', 'communicabledisease2_2011T', 'caste_grouped', 'caste']


In [21]:
master = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2011T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2011T'], df['communicabledisease2_2011T'],
     df['mentalhealth_2011T'],], axis=1)

In [22]:
darchula = pd.concat([master.ix['Darchula'][:16], master.ix['Darchaula'][16:]], axis=0)
darchula.name = 'Darchula'
master.drop(['Darchula', 'Darchaula'], inplace=True)
master = master.append(darchula, ignore_index=False)

kavre = pd.concat([master.ix['Kavrepalanchok'][:16], master.ix['Kavre'][16:]], axis=0)
kavre.name = 'Kavre'
master.drop(['Kavre', 'Kavrepalanchok'], inplace=True)
master = master.append(kavre, ignore_index=False)

master.to_csv('Data/master.csv')

## Create Master 2009-2012

In [23]:
master2009 = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2009T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2009T'], df['communicabledisease2_2009T'],
     df['mentalhealth_2009T'],], axis=1)
master2009 = master2009.rename(columns={'index':'District'})
master2011 = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2011T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2011T'], df['communicabledisease2_2011T'],
     df['mentalhealth_2011T'],], axis=1)
master2011 = master2011.rename(columns={'index':'District'})

In [24]:
master2009 = master2009.rename(columns={'BCG':'BCG3', 'DPT-HepB 1':'DPT-HepB 1 3', \
            'DPT-HepB 2':'DPT-HepB 2 3', 'DPT-HepB 3':'DPT-HepB 3 3', 'Japenese Encephalitis':\
        'Japenese Encephalitis 3', 'Measles 12-23':'Measles 12-23 3', 'Measles 9-11 months':\
        'Measles 9-11 months 3', 'Polio1':'Polio13', 'Polio2':'Polio23', 'Polio3':'Polio33',\
        'TT 1':'TT 1 3', 'TT 2+':'TT 2+ 3', 'TT2':'TT23','Confirmed Meningitis':\
        'Confirmed Meningitis 3', 'HIV/AIDS':'HIV/AIDS3', 'Leprosy':'Leprosy3', \
        'STD/STI':'STD/STI3'})

In [25]:
master0911 = pd.concat([master2009, master2011])

In [26]:
darchula = pd.concat([master0911.ix['Darchula'][:16], master0911.ix['Darchaula'][16:]], axis=0)
darchula.name = 'Darchula'
master0911.drop(['Darchula', 'Darchaula'], inplace=True)
master0911 = master0911.append(darchula, ignore_index=False)

kavre = pd.concat([master0911.ix['Kavrepalanchok'][:16], master0911.ix['Kavre'][16:]], axis=0)
kavre.name = 'Kavre'
master0911.drop(['Kavre', 'Kavrepalanchok'], inplace=True)
master0911 = master0911.append(kavre, ignore_index=False)

In [27]:
master0911.to_csv('Data/master0911.csv')

## Create Master 2009-2014 Dataset

In [28]:
master2009 = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2009T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2009T'], 
     df['mentalhealth_2009T'],], axis=1)
master2009 = master2009.rename(columns={'index':'District'})
master2009 = master2009.drop('Alcoholism',axis=1)
master2011 = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2011T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2011T'], 
     df['mentalhealth_2011T'],], axis=1)
master2011 = master2011.rename(columns={'index':'District'})
master2011 = master2011.drop('Alcoholism',axis=1)
master2013 = pd.concat(
    [grouped, df['poverty1T'], df['poverty2T'], df['vaccinations_2013T'],
     df['watersanitationT'], df['hdi2011'], df['sexratio'],
     df['communicabledisease1_2013T'], 
     df['mentalhealth_2013T'],], axis=1)
master2013 = master2013.rename(columns={'index':'District'})

In [29]:
master2009 = master2009.rename(columns={'BCG':'BCG3', 'DPT-HepB 1':'DPT-HepB 1 3', \
            'DPT-HepB 2':'DPT-HepB 2 3', 'DPT-HepB 3':'DPT-HepB 3 3', 'Japenese Encephalitis':\
        'Japenese Encephalitis 3', 'Measles 12-23':'Measles 12-23 3', 'Measles 9-11 months':\
        'Measles 9-11 months 3', 'Polio1':'Polio13', 'Polio2':'Polio23', 'Polio3':'Polio33',\
        'TT 1':'TT 1 3', 'TT 2+':'TT 2+ 3', 'TT2':'TT23','Confirmed Meningitis':\
        'Confirmed Meningitis 3', 'HIV/AIDS':'HIV/AIDS3', 'Leprosy':'Leprosy3', \
        'STD/STI':'STD/STI3'})

In [30]:
master2013 = master2013.rename(columns={'BCG5':'BCG3', 'DPT-HepB 1 4':'DPT-HepB 1 3', \
            'DPT-HepB 2 4':'DPT-HepB 2 3', 'DPT-HepB 3 4':'DPT-HepB 3 3', 'Japenese Encephalitis 4':\
        'Japenese Encephalitis 3', 'Measles 12-23 4':'Measles 12-23 3', 'Measles 9-11 months 4':\
        'Measles 9-11 months 3', 'Polio14':'Polio13', 'Polio24':'Polio23', 'Polio34':'Polio33', \
        'TT 1 5':'TT 1 3', 'TT 2+ 5':'TT 2+ 3', 'TT24':'TT23', 'Confirmed Meningitis 5':\
        'Confirmed Meningitis 3', 'HIV/AIDS5':'HIV/AIDS3', 'Leprosy5':'Leprosy3', \
        'STD/STI5':'STD/STI3'})
master2013 = master2013.drop(['HIV/AIDS', 'STD/STI'],axis=1)

In [31]:
master0913 = pd.concat([master2009, master2011, master2013])

In [32]:
master0913.to_csv('Data/master0913.csv')

# Calculate Per Capita

In [33]:
def per_capita(df):
    master = pd.read_csv('Data/'+df, index_col='Unnamed: 0')
    
    # Remove unnecessary columns
    master =  master.drop(
        ['Population', 'Standard Error of Poverty Gap', 'Standard Error of Poverty Incidence', 
         'Standard Error of Poverty Severity', 'Total Population', 'Zone', 'Geographical Region', 
         'Development Region', 'Zone.1', 'Geographical Region.1', 'Development Region.1'], axis=1)

    # Convert columns to float
    master['Percentage of People not expected to survive age 40'] = master['Percentage of People not expected to survive age 40'] * .01
    master['Percentage of children under age five who are malnurished'] = master['Percentage of children under age five who are malnurished'] * .01
    master['Percentage without safe water'] = master['Percentage without safe water'] * .01

    # Calculate per capita rates of disease/illness
    master.iloc[:,range(25, 38)] = master.iloc[:,range(25, 38)].div(master['Total Pop Census'],axis=0)
    master.iloc[:,range(42, len(master.columns))] = master.iloc[:,range(42, len(master.columns))].div(master['Total Pop Census'],axis=0)

    master['Vacc_Access'] = master[['BCG3', 'DPT-HepB 1 3', 'DPT-HepB 2 3', 'DPT-HepB 3 3', 
                                           'Japenese Encephalitis 3', 'Measles 12-23 3',
                                           'Measles 9-11 months 3', 'Polio13', 'Polio23', 'Polio33',
                                           'TT 1 3', 'TT 2+ 3', 'TT23',]].sum(axis=1)
    master['Vacc_Access'] = (master['Vacc_Access'] - master['Vacc_Access'].mean()) / master['Vacc_Access'].std()

    # Rename columns
    master = master.rename(
        columns={'Percentage of People not expected to survive age 40':'People not expected to survive age 40',
                 'Percentage of children under age five who are malnurished':'Children <5 years malnourished',
                 'Percentage without safe water':'Without safe water'})

    master = master.fillna(0)
    master.to_csv('Data/'+str(df[:-4])+'.csv')

In [34]:
per_capita('master.csv')
per_capita('master0911.csv')
per_capita('master0913.csv')

# Simplified Data

In [34]:
master = pd.read_csv('Data/master.csv')
master.columns

Index([u'Unnamed: 0', u'Foreigner', u'Hill Brahman', u'Hill Chhetree',
       u'Hill Dalit', u'Madhesi Brahman/Chhetree', u'Madhesi Dalit',
       u'Madhesi Other - OC A', u'Madhesi Other - OC B',
       u'Mtn/Hill Janajati A', u'Mtn/Hill Janajati B', u'Musalman', u'Newar',
       u'Other (MPB)', u'Other undefined', u'Tarai Janajati',
       u'Total Pop Census', u'Poverty Gap', u'Poverty Incidence',
       u'Poverty Severity', u'Adult illiteracy rate',
       u'Deprivation in economic provisioning', u'HPI',
       u'People not expected to survive age 40',
       u'Children <5 years malnourished', u'Without safe water', u'BCG3',
       u'DPT-HepB 1 3', u'DPT-HepB 2 3', u'DPT-HepB 3 3',
       u'Japenese Encephalitis 3', u'Measles 12-23 3',
       u'Measles 9-11 months 3', u'Polio13', u'Polio23', u'Polio33', u'TT 1 3',
       u'TT 2+ 3', u'TT23', u'Sanitation Coverage (%',
       u'Water Coverage Supply (%)', u'HDI ',
       u'Sex Ratio (male per 100 female)', u'Confirmed Meningitis 3',


In [35]:
def simplified(df):
    master = pd.read_csv('Data/'+df, index_col='Unnamed: 0')
    castes = np.square(master.iloc[:,[i for i in range(0,15)]])
    master['caste_diversity'] = castes.sum(axis=1)

    master[['BCG3', 'DPT-HepB 1 3', 'DPT-HepB 2 3', 'DPT-HepB 3 3',
            'Japenese Encephalitis 3', 'Measles 12-23 3',
            'Measles 9-11 months 3', 'Polio13', 'Polio23', 'Polio33', 'TT 1 3',
            'TT 2+ 3', 'TT23',]].sum(axis=1)

    simplified_df = master.drop(['Total Pop Census', 'Poverty Gap', 'Poverty Incidence',
                             'Deprivation in economic provisioning', 'HPI',
                             'People not expected to survive age 40', 'Without safe water',
                             'Water Coverage Supply (%)', 'HDI '], axis=1)
    
    simplified_df['Vacc_Access'] = master[['BCG3', 'DPT-HepB 1 3', 'DPT-HepB 2 3', 'DPT-HepB 3 3', 
                                           'Japenese Encephalitis 3', 'Measles 12-23 3',
                                           'Measles 9-11 months 3', 'Polio13', 'Polio23', 'Polio33',
                                           'TT 1 3', 'TT 2+ 3', 'TT23',]].sum(axis=1)
    simplified_df['Vacc_Access'] = (simplified_df['Vacc_Access'] - simplified_df['Vacc_Access'].mean())/simplified_df['Vacc_Access'].std()

    simplified_df.drop(['BCG3', 'DPT-HepB 1 3', 'DPT-HepB 2 3', 'DPT-HepB 3 3', 
                        'Japenese Encephalitis 3', 'Measles 12-23 3', 'Measles 9-11 months 3',
                        'Polio13', 'Polio23', 'Polio33', 'TT 1 3', 'TT 2+ 3', 'TT23',], axis=1, inplace=True)
    simplified_df.to_csv('Data/simplified_'+str(df[:-4])+'.csv')

In [36]:
simplified('master.csv')
simplified('master0911.csv')
simplified('master0913.csv')